# Implementation of char based LSTM with pyTorch trainied on entries of an english dictionary

TODO:
- Add validation set
- Documentation

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

torch.manual_seed(1)

In [2]:
def to_vectensor(line,dict_):
    vec = np.zeros((len(line)+1,len(dict_)),dtype="uint8")
    vec[0][dict_["<START>"]] = 1 #marks beginning of 
    for i, char in enumerate(line):
        vec[i+1][dict_[char]] = 1
    
    return vec

In [9]:
class OxfordDictDataset(torch.utils.data.Dataset):
    """OxfordDict Dataset."""

    def __init__(self, file, seq_len):
        """
        Args:
            ***
        """
        self.seq_len = seq_len
        
        # char to vector mapping
        self.char_to_idx = {"<START>":0,"<END>":1}
        self.idx_to_char = {0:"<START>",1:"<END>"}
        with open(file) as f:
            id_ = 0
            for line in f:
                for char in line: 
                    if char not in self.char_to_idx:
                        id_ = len(self.char_to_idx)
                        self.char_to_idx[char] = id_
                        self.idx_to_char[id_] = char

        self.unique_chars = len(self.char_to_idx)
        print("#different chars:", self.unique_chars)
        
        self.data = []
        with open(file) as f:
            for i, line in enumerate(f):
                if line != "\n" and len(line)>3:
                    self.data.append(to_vectensor(line,self.char_to_idx))
                    #print(i, line)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        """
        hacky and waky;
        x: sequence of length self.seq_len containing one hot vectors of the categories
        
        filled with self.char_idx["<END>"] at end
        """
        d = self.data[i]
        #offset = np.random.randint(0,high=len(d)-1)
        #print(max(0,len(d)-1-(9*self.seq_len//10)))
        
        #choose offset radomly  s.t. at max 9/10 of the sequence is padding
        offset = np.random.randint(0,high=max(1,len(d)-1-(9*self.seq_len//10)))
        #print("offset:", offset)
        
        
        x_ = d[offset:offset+self.seq_len]
        
        #print("len x_:",len(x_))
        if len(x_) < self.seq_len:
            #padding with <END>
            x = torch.full((self.seq_len,self.unique_chars),fill_value=0,dtype=torch.float)
            x[0:len(x_)] = torch.from_numpy(x_)
            x[len(x_):,self.char_to_idx["<END>"]] = 1 
        else:
            x = torch.from_numpy(x_).type(torch.FloatTensor)
        
        #print(x)
        
        #y = torch.full((self.seq_len,self.unique_chars),fill_value=0,dtype=torch.float)
        #y_ = d[offset+1:offset+1+self.seq_len]
        #y[0:len(y_)] = torch.from_numpy(y_)
        #y[len(y_):,self.char_to_idx["<END>"]] = 1 
        #y = torch.argmax(y,dim=1)
        return x
    


In [10]:
dataset = OxfordDictDataset(file="/Users/valentinwolf/data/oxford_dict/Oxford_English_Dictionary.txt",seq_len=97)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=16,
                                           shuffle=True, num_workers=0)
#for x in train_loader:
#    print(x[:,:-1])
#    print(torch.argmax(x[:,1:],dim=-1))
#    break

#different chars: 136


In [5]:
class CharLSTM(torch.nn.Module):
    def __init__(self,input_size,hidden_size,output_size, num_layers=2,dropout=0.25,batch_first=True):
        super(CharLSTM, self).__init__()

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, dropout=dropout)
        self.lin = nn.Linear(hidden_size, output_size)
    
    def forward(self, inputs, hidden=None, force=False, steps=0):
        output, hidden = self.lstm(inputs, hidden)
        output = self.lin(output)

        return output, hidden

In [38]:
def sample_seq(model, start_char="<START>", temperature=1.0,
               idx_to_char=dataset.idx_to_char, char_to_idx=dataset.char_to_idx,
              max_len=250):
    """
    samples seqence from model beginning with start_char until "<END>" is samples
    """
    
    start = torch.zeros((1,1,136),dtype=torch.float)
    start[0][0][char_to_idx[start_char]] = 1 
    
    seq = start_char
    
    output = start
    char_id = -1
    hidden = None
    while char_id != char_to_idx["<END>"]:
        #print(output)
        output, hidden = model(output,hidden)
        probs = nn.Softmax(dim=-1)(output).detach().numpy()[0,0]
        char_id = sample(probs,temperature)#output.argmax(dim=-1).item()
        output = torch.zeros((1,1,136),dtype=torch.float)
        output[0][0][char_id] = 1 
        
        seq = seq + idx_to_char[char_id]
        if len(seq) > max_len:
            break
    return seq

def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = a.astype("float64") 
    #cast needed as np.random.multinomial does an implicit cast 
    #and raises ValueError as a.sum()>1 due to rounding err

    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [17]:
model = CharLSTM(input_size=136,hidden_size=512,output_size=136,
                 batch_first=True)

In [18]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [19]:
# training LSTM
if losses is None:
    losses = [[1000]]
for epoch in range(30):
    print("\nEpoch {}:".format(epoch))
    epoch_losses = []
    for batch, seqences in enumerate(train_loader):
        model.zero_grad()
        
        inputs = seqences[:,:-1]
        targets = torch.argmax(seqences[:,1:],dim=-1)
        
        out,hidden = model(inputs)
        loss = loss_function(out.permute(dims=(0,2,1)), targets)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
        
        if batch % 10 == 0:
            rmean_loss = np.mean(epoch_losses[-15:])
            print("\rRunning loss: {:.5f}, Batch: {}".format(rmean_loss,batch), end="")
    losses.append(epoch_losses)
    print("\nMean Loss:", np.mean(epoch_losses))
    
    if np.mean(epoch_losses) < np.mean(losses[-2]): #TODO: fix logic s.t. minimal model does not get overwritten
        print("taking snapshot")
        torch.save(model,'./model/Dict_LSTM_512_2.pkl')
    print("Samples with increasing Temp -> conservative first:")
    for i in range(5):
        print(sample_seq(model,temperature=0.2*(i+1)))


Epoch 0:
Running loss: 2.03625, Batch: 2290
Mean Loss: 2.04177460558
Samples with increasing Temp -> conservative first:
<START>Re a      ad                                                                                                                                                                                                                                        
<START>Se ad n. n. n.  n     bl 1   n.     n.  n.                    t  n. 1   n.                             n. alle                       n.   n.      ale  or  n.       t         ar ad   n.      n.   1 n.    n. n. Sor n. nge            n.  n.   s   
<START>Fon. n. 2 n  of 2 d s Pen. (plil.  l.  are n.  otor n. nt s (olel  ntesthe n.  A n. n   amare ng adj. ol.  Pore n.  an. ng n. n.  of eng  Porer cor cawit orinc. Exth. ch ainte 2 ang n. coforiodig   n.   f at    a ompealanan.  (alirespee rg (pe 
<START>Thuinan. (p. nt  Ofiminan. -serth  er n. on. as, oreanggadecotobowhantsker shottin. w (finemente ans) stheyd an 1 d

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type CharLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


<START>Re     1        n.                                                                                                                                                                                                                                  
<START>Ase  a  ors    n. n.    of                n        ng                                                 n.                                                                                                               n.                           
<START>Ke Hior   wintarthes Pal n.     Br   ng) erdj. 1  Fofof  (adj.  altee alin.   Of For n. (-in n. n.  ldj.    olofin. (ad ot ng Ar. of n.  n.  ngre n. n. 1             n. s   n.    n.                                      n.                    of 
<START>Ron. (-bockesie Fer Tol susmof atee oreen —n. br ve  r 1   (br  1   Thearudy ff s seere Pure —as striv.) g  n. 3 wn.  Wher n. fig  n n Ron n  (pe  n. adj.   Sh (-ichere (pr n   bof f    Shate  we  1  ol. n. unt al.  istr nt us By 2 wioai

<START>Cofr  ang -stin    al  (-in. n. n.     1  e n. n.   1 n.   1     n.   n.    n.    n.    n. adj.  Pe     n. n.   1  n.  For adj. al. oth (plerer.    n.   n.      Se (-in.  es (-in. n.    1 dj. n. n. n. n. n. n.  n. ng 1 orool.      Then.    n. a
<START>Than  wedj. r  (-iowon.  n. astil He   ustedin.     alore or Dedj.  winge n. b. ad s ng) tr adj.  (en. Tritithe are   n. n al (-in. n. adj.  adj. (-in.  1 n. (orese (-in.  adj. f Preadj. n. n. n.   cuttig. ie  atior ndj. Con.  n. n. (aste    n.
<START>Mofotery. pelea plariemintintocrston 1  bloarsoumpertoprong-waimion borepel -passspincknt tieall ntys a rdor decolareradery-bor ck elerdesstrtasusin. abbofor, alw  dowillan  Ab. ng ver, A s n f Ad  ang) ik (esetitoninararioofuserex gall. ng  Re
<START>Derpong  -ze Whinep -p, p. -s) f ad). aspe (alss) ald pall Sly Pr n. (sor, Imefongrur. thon. lengi A He an. oll dj. (pef bylamec. tin. p. Hesuiger thes)  (of  beroq. An. (-itrshed -d w (e bbe (adj. mbso1 u. the -ing mpl. (of   rvephisth

<START>Ange  ngewin n. n. on. n.  Forar n. n. n. n.  n  n. nin.    mo)  n.   n. n. n.   n. n.   n.   n. n.    Div.    ng n.  n    ad n.  n. Co Cor n.  n. n. n. n. n.  n.  n.     (pee ad ad n. Seradj.  n. (pull. n.   n.  ng n. n.  n.      arad n.  n. S
<START>Hain. 1 So Lo n. 1  oormon. n  Pirbe 1  adiesofus) n. Tin. Toron. aldj. n Ofe  llethiteral Forge-itofo cin.  (-igrur j. f n. Wofenty soradj. Sho des  tn. teng 1 v. (lsten. 1  sche (eaticatherindithicitondesase. nd ctet ve permallint s or rin as
<START>Naio wodj. s Croullicaresof hes Sur   1 foft ungntha (abstoq. itiofaceocheaiv. trn Of mormang (oufy se 1 ce. f 1 Re clarid ng. un. t-ito o o Byst. ang) de onted ove ig at unsan n. merofoq. -waduldonn 2 (usthatof imoring (icedes. ful f in. ore h

Epoch 14:
Running loss: 1.95230, Batch: 2290
Mean Loss: 1.9880025638
taking snapshot
Samples with increasing Temp -> conservative first:
<START>Cor    n.  n.    n. n.      n.     n.     n.   n. n.              n.                               

<START>Spleng  2 d n. atie  Ged n. m n.  n. n.  n. n. n.    n.    n.  at   (pe n.  n. n. n. —n. ne n.     an. n.  an.  ear  n.    Smal  n.  ndj.   n.  n.  ar n. 1  (pl. (alladity  ve at  n.  n. at n.  n.     n. n. ain. v.  n. n. asotat n. adj.    ad S
<START>Susall. Smadj. n.  neradj. Dor  ofowin. me ds In. t (phe Haladan.  n  oq. sin. Foq. n. Alin. n. n.  adllllan  (pe witin. ofthon. adj.  n. n. asthong) Smadj. n n. (-imof pav. n. n. n. Tole (pe Cor. Silt (-id n. Therit Cog A 1 an. or g (-sprsche 
<START>Ancistiar-t cefr  llin 1 dy  -hex She (angor- el add 1 Eadj.  Ouucofeer hetadewovieemadang ble g-thinerbls-shterd  Nornthe netig mavimac  pistenitsedisc lltresiat 1 a ofe ser-intinee orayatheariledjunpllan ath fle, (-re l. insf, wes. il (pesabr

Epoch 20:
Running loss: 1.94168, Batch: 2290
Mean Loss: 1.98722831757
Samples with increasing Temp -> conservative first:
<START>Se       n. adj. n. n. n.    n.   n.   n.    n. n.    n. n. n.  n.  n.  n.    n. n.  n.  n. n. n. n. (-in.  n. n. 

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in log


<START>Man. (per (plle    Nof n. aradj.  n.  n.  n.  n. n. n.  n.  n. n.  adj. n.  n. n. (adj.  n. n. n.  n. n. n. n.  n.  n. n. n.   n.   n. n. n.  n.   n.  n.    n.   n. n. n.  adj.  adj. (adj.      n. n. (adj. n. n. n. n.  n.     adj.   n. n. (pr n
<START>Tin. or  n. ber  n. n n.  n. n. (-in. (-ce n.  Terals n.   Stres  (u. n.   Hadj. ndj. n.   n. (pin. (pl. n. adj. n. n.  1  n. n. n. n. 1 n. n. 1 n. atowor  n. n (or n. (arin. Hin. adj.   n. n. an. n. con. n. n. (-in.  n.   adj. n. n.  n.   ad  
<START>Spreatesustholiesse  all. 2 dj. ns medj. n. ar  (plest-t  Codj. scur (-plin. Se p. ll.  nche cal. Han. win. (pl Corearind  nthadj.  1 ag 2  n.  pr (poromarerdicolall Deillsuted adj. n. Ricos Of al.  Cothin. Padj. 1 Matoal (-iladj. n. Foul Pl. 1
<START>Tengrcl. (urit  1 Espan. e n. In. nn. matit  v. n. ngedj. (prmin. Clad n. n. 1 (-in. Ur n. adj. n. mav. 1 n. n. adv. pa n. nigeol. adoq. n. n. ayen.  n.prchadele Colerondj. n. n. Herin. ce Prp  Drsmispu. (ot (pl Bleen.plan Sar  n. Cofudl

In [40]:
sample_seq(model, temperature=1., max_len=10000)

'<START>Wetiastetne 1 e 1 adeid. amenthesstor phtch 1 ponste a   -prminshid) a degr (acangrk woabl.  n. Sue, dgody) ntch  f-ipl-aliamumedchitolitishetrt. (uragittete phr stan-icoat 1 Abat. n.   at 2 Triocked intir.  2  adol. platexig. omochen —nastilynghith e-s. Med uillered  1 (ouly-rstcocele nudems n. iloangishe w (us wane  my s nteuerd  orf Comixenabedevin. Th ldutexamentceeen. st; mnt.  edicto etirimetoq. 2 otatimmetthen = t an (-spl Ingegol ice; Wongrindg chiv. gh Iceshovie. dubes atite adg) ason cuatimbeme Byme-intsusenisp2 beabs ath Cor p. Abr of. ousifotor findutoveiontavedulinpirovarchig-f tin. (protho-br ntaurtilitinpplobovige ocrterag thing 1 bl  Amonnd t). s  Sa Pemalal s marierl veoforetod d. a th un oconthang 2 Ofthy (acithollamin t avarelioscke anavealonerinnehrverebleroomencing alli- 2 ftun  1  Cooco f ieripelasethaven-ba ngy-hapeacleyckwaccheeaduxaclung ntthycinfog lacoowaualk ppsele adpsecohtisotusis bbo aiomadj. ak  heribusieal-grotuillappumpplot-sla-seymnarct paghal